__Models__ 
* This file trains two different models. The first one make long term prediction by taking into account how traffics ebbs and flow over the period of one year. This takes advantage of hourly vechile counts collected over several year at major intersections around Adelide CBD. It also takes into account weather (rain, temp, humidity, etc)
* The second model predicts hourly traffic in the city using some of the above data along with bluetooth becons that end up in the city as their destination. This actually measure how many cars are parked in the city, as opposed to how many move throught the city.

In [1]:
import pandas as pd

#For the learning bit 
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing



In [2]:
df = pd.read_csv('intersection_data.csv')
location = df.groupby('site_no')['lat','lon'].mean()
location.to_csv('locations.csv')

df['day_of_year'] = pd.DatetimeIndex(df['rec_date']).dayofyear
df['hour_of_day'] = pd.DatetimeIndex(df['rec_hour']).hour



In [3]:
hourly_traffic_by_location = df.groupby(['site_no','day_of_year','hour_of_day'])['hourly_traffic']
avg_traffic = pd.DataFrame(hourly_traffic_by_location.mean().reset_index(name = "hourly_traffic_avg"))
avg_traffic.to_csv('data_clean')



In [60]:
#y = preprocessing.scale(avg_traffic['hourly_traffic_avg'])

df_w = pd.read_csv('weather_by_day_of_year.csv')
df_all = pd.merge(avg_traffic, df_w , on = ['day_of_year'])

y = np.array(df_all['hourly_traffic_avg'])
X = df_all[['site_no', 'day_of_year','hour_of_day', 'rain','max_temp','min_temp','max_humid','min_humid','wind','solar']]

df_training_data = df_all[['site_no', 'day_of_year','hour_of_day', 'rain','max_temp','min_temp','max_humid','min_humid','wind','solar','hourly_traffic_avg']]
df_training_data.to_csv('Model_Year_training_data.csv')


scalerX = preprocessing.MinMaxScaler()
scalerY = preprocessing.MinMaxScaler()

X = scalerX.fit_transform(X)
y = y/np.max(y)

clf = linear_model.SGDRegressor(max_iter=1000, tol=1e-3)
clf.fit(X, y)


C:\Users\ylatif\Miniconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=1000,
       n_iter=None, n_iter_no_change=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, tol=0.001, validation_fraction=0.1,
       verbose=0, warm_start=False)

In [61]:
d = clf.predict(X)-y
print("Test error MSE : ", np.matmul(d.T,d)/len(y))

Test error MSE :  0.020480126183113193


In [23]:


dfa.head()

,Unnamed: 0,site_no,hour_of_day,DestId,index
0,0,3001,0,3001,74
1,1,3001,1,3001,41
2,2,3001,2,3001,58
3,3,3001,3,3001,83
4,4,3001,4,3001,86


In [13]:
dfa = pd.read_csv('hourly_hits_by_gps_location2.csv')
this = avg_traffic.groupby(['site_no','hour_of_day'])['hourly_traffic_avg'].mean().reset_index()
this.to_csv('hourly_traffic_count_per_site.csv')

In [33]:
merged = pd.merge(this, dfa, on=['site_no','hour_of_day'], how='outer')

In [52]:

training_data = merged[['site_no','hour_of_day','index']]
training_data = training_data.fillna(0)

objective = merged['hourly_traffic_avg']

df_training_data_hourly = merged[['site_no','hour_of_day','index','hourly_traffic_avg']]
df_training_data_hourly.to_csv('Model_Hourly_training_data.csv')


y = np.array(objective)
X = training_data

scalerX = preprocessing.MinMaxScaler()
scalerY = preprocessing.MinMaxScaler()

X = scalerX.fit_transform(X)
y = y/np.max(y)

clf = linear_model.SGDRegressor(max_iter=1000, tol=1e-3)
clf.fit(X, y)


C:\Users\ylatif\Miniconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=1000,
       n_iter=None, n_iter_no_change=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, tol=0.001, validation_fraction=0.1,
       verbose=0, warm_start=False)

In [53]:
d = clf.predict(X)-y
print("Test error MSE : ", np.matmul(d.T,d)/len(y))

Test error MSE :  0.026355135825672612
